Exercise : MT Inversion\
Class    : Physics for Geosystems (M.Sc. Track GeoEnergy Engineering)

Department of Geoscience and Engineering\
Delft University and Technology


Created by La Ode Marzujriban Masfara © : l.o.m.masfara@tudelft.nl

In [2]:
#load all necessary packages
import numpy.matlib
import matplotlib.pyplot as plt
import numpy as np 
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")

# Problem setup

In this notebook, you are asked to do forward modeling for a specific MT and use the generated data to get the MT back using the full-waveform inversion technique. As discussed in the lecture, to model seismograms from specific MT, we must compute Green's functions beforehand to generate elementary seismograms. For this exercise, we assume that the elementary seismograms are known. The source location and receivers of these elementary seismograms are depicted in the figure below. The red star represents the source location, and each of the white triangles corresponds to the location of the receivers. In short, you are asked to do the following steps:

1. Create functions to convert from expansion coefficient __a__ to __M__ and vice versa. 
2. Define matrix __M__ for a strike-slip fault movement and use the function you create in step 1 to get the corresponding __a__. 
3. Next, use what you obtain from step 2 to generate the affiliated seismograms and plot them in the notebook. 
4. Now, define a random __M__ and use it to generate seismograms.
5. Set up your inverse problem.
6. Use the seismograms you obtain from step 4 to invert for your pre-defined __M__.


bonus points:
Generate random noise and use them to corrupt your forward modeled data and use it for the inversion. Discuss what you observe. 

7. Summarize what you have done in the summary tab (#7). Particularly for step 3, discuss what you observe in the seismogram first-arrival polarities and elaborate on why you get the corresponding polarities. 

Save your notebook and change the file name to your name and student number (NAME_STUDENTNUMBER.ipynb)

![title](studycase_001.png)

# Load data (time array and elementary seismogram)

Here we load the elementary seismogram given the estimated source location to each of the receivers. 


In [3]:
#load elementary seismogram & time array
elementaryseismo = np.load('elementaryseis.npy')
timearray = np.load('timearray.npy')

Based on the expansion coefficients, displacements can be modeled using the relation below:
$$
u_{i}\left(\mathbf{x}_{r} ; \mathbf{x}_{a}, t\right)=\sum_{n=1}^{6} a_{n} S_{i}^{n}\left(\mathbf{x}_{r} ; \mathbf{x}_{a}, t\right)
$$

In [4]:
elementaryseismo.shape

(6934, 6, 3, 10)

Here is the detail of the elementaryseismo array:

- The first dimension corresponds to the number of samples in time (6934 sample)
- The second dimension corresponds to elementary seismogram (1-6)
- The third dimension is the recorded components (x,y,z)
- The fourth dimension is the receivers (see figure in the problem setup)

# 1 Functions AtoM & MtoA

The following is the transfer function from A to M:


$$
\mathbf{M}=\sum_{n=1}^{6} a_{n} \mathbf{M}^{n}=\left(\begin{array}{ccc}-a_{4}+a_{6} & a_{1} & a_{2} \\ a_{1} & -a_{5}+a_{6} & -a_{3} \\ a_{2} & -a_{3} & a_{4}+a_{5}+a_{6}\end{array}\right)
$$

# 2 Define M for a strike-slip fault movement 

# 3 Seismogram due to a strike-slip fault

Based on the expansion coefficients, displacements can be modeled using the relation below:
$$
u_{i}\left(\mathbf{x}_{r} ; \mathbf{x}_{a}, t\right)=\sum_{n=1}^{6} a_{n} S_{i}^{n}\left(\mathbf{x}_{r} ; \mathbf{x}_{a}, t\right)
$$

Given MT matrix for the strike-slip fault movement, the focal mechanism and radiation pattern for this fault are shown in picture below:
![title](Addition.png)

As discussed in the class, the white areas in focal mechanisms represent dilatation, whereas the black areas represent compression. The center of the focal mechanism represents the epicenter. Therefore for the focal mechanism above, receivers located in the black shaded area will have positive polarity in their P-wave first arrivals and vice versa for the white shaded area. If the first-arrival polarities are not discernible, one can observe systematic polarity flips between the seismograms from receivers located in black areas against seismograms from receivers in white shaded areas as indicated by the radiation pattern. 

# 4 Define a random M

In [5]:
#FUNCTION gensignal will generate seismograms given expansion coefficients (A)
def gensignal(A):
    
    #create container for the seismograms
    seismogram = np.zeros((6934,3,10))
    
    #iterate over recorded components
    for i in range(3):
        
        #iterate over receiver
        for j in range(10):
            
            #take elementary
            S = elementaryseismo[:,:,i,j]
            
            #remap A
            Amatrix = numpy.matlib.repmat(A,6934,1)
            
            #elementwise multiplication of A and Ssum over column
            AmultS = np.multiply(Amatrix,S)
            
            #now sum over column
            AmultS = np.sum(AmultS,axis=1)
            
            #collect seismogram 
            seismogram[:,i,j] = AmultS
            
    return(seismogram)

In [6]:
#random A
# you can use functions that you have created (MtoA) to obtain the A from your random M
A = np.array((100,500,100,-190,100,-100))

In [10]:
#generate seismograms from random A
#creates seismograms recorded in three components from 10 receivers.
seismo = gensignal(A)

In [8]:
seismo.shape

(6934, 3, 10)

# 5 Construct the Inverse problem matrices.

Here, we use the seismograms generated from the random M above as if it were (real) observed data (d$^{obs}$), which means (as if) the moment tensor is not known. Given the scenario, to obtain M, we can use a full waveform inversion technique by minimizing the difference between observed data (d$^{obs}$) and forward modeled data, Gm. G is the forward modeling operator, which is, in this case, are the elementary seismograms. Furthermore, m is the model parameter which are the expansion coefficients for our case. Accordingly, by varying m(A), any seismogram can be generated using the same G.

In [11]:
#we use only seismogram from one of the recording components (x) from one receiver 
G = elementaryseismo[:,:,0,0].T

#seismogram of receiver 1 for x-component
dobs = seismo[:,0,0]

# 6 solve the Inverse problem

Misfit function: 
$$
\chi(\mathbf{m})=\frac{1}{2}\left(\mathbf{d}^{\mathrm{obs}}-\mathbf{G m}\right)^{T}\left(\mathbf{d}^{\mathrm{obs}}-\mathbf{G m}\right)=\frac{1}{2} \sum_{i=1}^{N}\left(d_{i}^{\mathrm{obs}}-\sum_{j=1}^{n} G_{i j} m_{j}\right)^{2}
$$

the solution 
$$
\mathbf{m}^{\text {est }}=\left(\mathbf{G}^{T} \mathbf{G}\right)^{-1} \mathbf{G}^{T} \mathbf{d}^{\text {obs }}
$$

In [18]:
#estimate the A
mest = (np.linalg.inv(G@G.T)@G)@dobs

print('estimated A ='+str(mest))
print('true A ='+str(mest))

estimated A =[ 100.  500.  100. -190.  100. -100.]
true A =[ 100.  500.  100. -190.  100. -100.]


# 7 Summary
put your summary here...